In [1]:
import basis_set_exchange as bse
import scipy as sp
import matplotlib.pyplot as plt
from pyscf import gto,scf
import copy
import numpy as np
import scipy
from scipy.interpolate import interp1d
from pyscf.data.elements import _symbol
from pyscf.grad import RHF as g
import inspect
import sys
sys.path.append("../Libs")
from FcMole import FcM,FcM_like
from FDcoeffs import get_coeffs
from ao_scf import ao_RHF
from numpy.linalg import norm

In [161]:
from functools import reduce
import ctypes
import numpy
from pyscf import lib
from pyscf import gto
from pyscf.lib import logger
from pyscf.scf import _vhf
from pyscf.scf import cphf

# import _response_functions to load gen_response methods in SCF class
from pyscf.scf import _response_functions  # noqa
# import pyscf.grad.rhf to activate nuc_grad_method method
from pyscf.grad import rhf  # noqa

In [4]:
def make_dP(mf,mo1):
    mol=mf.mol
    nao=mol.nao
    nocc=mf.mol.nelec[0]
    C=mf.mo_coeff
    dP=np.zeros_like(C)
    dP[:,:]=2*np.einsum('ij,jk,lk->il',C,mo1,C[:,:nocc])
    return dP+dP.T

def make_U(mo1):
    U=np.zeros((mo1.shape[0],mo1.shape[0]))
    U[:,:mo1.shape[1]]=mo1
    U=U-U.T
    return U

In [48]:
import inspect
import sys
sys.path.append("../Libs")
from FcMole import FcM
from FDcoeffs import get_coeffs
from ao_scf import ao_RHF

In [139]:
nn=gto.M(atom="N 0 0 0; N 0 0 2.05",unit="Bohr",basis=bse.get_basis("6-31g",fmt="nwchem",elements=[6,7,8]))
mf_nn=scf.RHF(nn)
e_nn=mf_nn.scf()
mf=mf_nn
P_nn=mf_nn.make_rdm1()
S_nn=mf_nn.get_ovlp()
h1_nn=mf_nn.get_hcore()
G_nn=nn.intor('int2e', aosym='s8')
########
on=gto.M(atom="O 0 0 0; N 0 0 2.05",unit="Bohr",basis=bse.get_basis("6-31g",fmt="nwchem",elements=[6,7,8]),\
           charge=1) # ON
mf_on=scf.RHF(on)
e_on=mf_on.scf()
P_on=mf_on.make_rdm1()
S_on=mf_on.get_ovlp()
h1_on=mf_on.get_hcore()
G_on=on.intor('int2e', aosym='s8')

########
cn=gto.M(atom="C 0 0 0; N 0 0 2.05",unit="Bohr",basis=bse.get_basis("6-31g",fmt="nwchem",elements=[6,7,8]),\
           charge=-1)# CN
mf_cn=scf.RHF(cn)
e_cn=mf_cn.scf()
P_cn=mf_cn.make_rdm1()
S_cn=mf_cn.get_ovlp()
h1_cn=mf_cn.get_hcore()
G_cn=cn.intor('int2e', aosym='s8')


converged SCF energy = -108.867942022953
converged SCF energy = -128.797709060453
converged SCF energy = -92.2174104946117


In [64]:
dS=(S_on-S_cn)/2
dh1=(h1_on-h1_cn)/2
dG=(G_on-G_cn)/2
dVnn=(8*7-7*7)/2.05
d2S=(mf_on.get_ovlp()+mf_cn.get_ovlp()-2*mf.get_ovlp())
d2h1=(mf_on.get_hcore()+mf_cn.get_hcore()-2*mf.get_hcore())
d2G=(G_on+G_cn-2*G_nn)
dG4d=(on.intor('int2e')-cn.intor('int2e'))/2

In [83]:
def E(l):
    return ao_RHF(FcM_like(nn,fcs=[l,0]),S_ao=S_nn+dS*l+d2S*l**2/2,hcore=h1_nn+dh1*l+d2h1*l**2/2,\
                  eri=G_nn+dG*l+d2G*l**2/2).scf()
def P(l):
    mf=ao_RHF(FcM_like(nn,fcs=[l,0]),S_ao=S_nn+dS*l+d2S*l**2/2,hcore=h1_nn+dh1*l+d2h1*l**2/2,\
                  eri=G_nn+dG*l+d2G*l**2/2)
    mf.scf()
    return mf.make_rdm1()
dP_fd=(P(.05)-P(-.05))/.1

converged SCF energy = -109.78516197393
converged SCF energy = -107.959073811474


In [9]:
import pyscf.hessian # needed to use hessian

In [84]:
# This is a part of the gradient 
0.5*np.einsum("ij,kl,ijkl",P_nn,P_nn,dG4d)-np.einsum("ik,jl,ijkl",P_nn,P_nn,dG4d)/4

1.8049740222041737

In [70]:
(np.einsum("kl,ijkl->ij",P_nn,dG4d)-np.einsum("jl,ijkl->ik",P_nn,dG4d)/2)

array([[ 1.16938752e+00,  2.69419668e-01,  7.42084859e-02,
         6.71411476e-18,  9.40318006e-18, -1.21628314e-02,
         8.78841208e-18,  1.02727691e-17,  6.27375160e-04,
         2.46108218e-04, -4.72027652e-02, -1.02305100e-01,
         7.27887758e-18,  8.41816511e-18,  9.06439338e-02,
         7.35799303e-18,  8.69061357e-18,  1.77648682e-01],
       [ 2.69419668e-01,  7.20219137e-01,  2.95356262e-01,
         2.18257440e-17,  1.91056283e-17, -6.61554194e-02,
         1.95651621e-17,  2.05520074e-17, -4.73085020e-02,
        -1.63113113e-01, -4.04435445e-01, -3.72215508e-01,
         2.01403867e-17,  2.38595150e-17,  1.93451833e-01,
         1.68761787e-17,  2.01823064e-17,  1.73626547e-01],
       [ 7.42084859e-02,  2.95356262e-01,  4.60796373e-01,
         1.01897571e-17,  4.99410443e-18, -6.33435929e-02,
         4.17688623e-18,  1.71549987e-18,  1.46610595e-02,
        -2.51506192e-02, -2.47381340e-02, -6.18243371e-02,
         7.91029201e-18,  1.19489014e-17, -3.28168553e

In [127]:
# qui h1ao e' la somma del contributo di h monoelectr + V eff (da operatori bielettronici)
h1_ao=dh1+np.einsum("kl,ijkl->ij",P_nn,dG4d)-np.einsum("jl,ijkl->ik",P_nn,dG4d)/2

In [134]:
numpy.einsum('pq,pq', h1_ao, dP_fd)

-0.7066089954695508

In [133]:
mol=nn
max_memory=4000
verbose=None
mo_energy = mf.mo_energy
mo_occ = mf.mo_occ
mo_coeff = mf.mo_coeff
mocc = mo_coeff[:,mo_occ>0]
atmlst = range(mol.natm)
log = logger.new_logger(h_nn, 1)
#mo1, mo_e1 = pyscf.hessian.rhf.solve_mo1(mf,mo_energy, mo_coeff, mo_occ, h1_ao,None, atmlst, max_memory, log)
#mo1_2, mo_e1 = pyscf.hessian.rhf.solve_mo1(mf,mo_energy, mo_coeff, mo_occ, h1ao,None, atmlst, max_memory, log)

Solve_mo1
    atmlst = range(mol.natm)
    nao, nmo = mo_coeff.shape
    mocc = mo_coeff[:,mo_occ>0]
    nocc = mocc.shape[1]

    fx = gen_vind(mf, mo_coeff, mo_occ)
    s1a = -mol.intor('int1e_ipovlp', comp=3)

    def _ao2mo(mat):
        return numpy.asarray([reduce(numpy.dot, (mo_coeff.T, x, mocc)) for x in mat])

    mem_now = lib.current_memory()[0]
    max_memory = max(2000, max_memory*.9-mem_now)
    blksize = max(2, int(max_memory*1e6/8 / (nmo*nocc*3*6)))
    mo1s = [None] * mol.natm
    e1s = [None] * mol.natm
    aoslices = mol.aoslice_by_atom()
    for ia0, ia1 in lib.prange(0, len(atmlst), blksize):   #solo per non andare fuori memoria
        s1vo = []
        h1vo = []
        for i0 in range(ia0, ia1):
            ia = atmlst[i0]
            shl0, shl1, p0, p1 = aoslices[ia]
            s1ao = numpy.zeros((3,nao,nao))
            s1ao[:,p0:p1] += s1a[:,p0:p1]
            s1ao[:,:,p0:p1] += s1a[:,p0:p1].transpose(0,2,1)
            s1vo.append(_ao2mo(s1ao))
            h1ao = h1ao_or_chkfile[ia]
            h1vo.append(_ao2mo(h1ao))

        h1vo = numpy.vstack(h1vo)
        s1vo = numpy.vstack(s1vo)
        mo1, e1 = cphf.solve(fx, mo_energy, mo_occ, h1vo, s1vo)
        mo1 = numpy.einsum('pq,xqi->xpi', mo_coeff, mo1).reshape(-1,3,nao,nocc)
        e1 = e1.reshape(-1,3,nocc,nocc)

        for k in range(ia1-ia0):
            ia = atmlst[k+ia0]
            mo1s[ia] = mo1[k]
            e1s[ia] = e1[k].reshape(3,nocc,nocc)
        mo1 = e1 = None
    else:
        return mo1s, e1s


In [141]:
def _ao2mo(mat):
    return np.asarray([reduce(numpy.dot, (mo_coeff.T, x, mocc)) for x in mat])
h1vo=_ao2mo(h1_ao)
s1vo=_ao2mo(dS)

In [163]:
def gen_vind(mf, mo_coeff, mo_occ):
    nao, nmo = mo_coeff.shape
    mocc = mo_coeff[:,mo_occ>0]
    nocc = mocc.shape[1]
    vresp = mf.gen_response(mo_coeff, mo_occ, hermi=1)
    def fx(mo1):
        mo1 = mo1.reshape(-1,nmo,nocc)
        nset = len(mo1)
        dm1 = numpy.empty((nset,nao,nao))
        for i, x in enumerate(mo1):
            dm = reduce(numpy.dot, (mo_coeff, x*2, mocc.T)) # *2 for double occupancy
            dm1[i] = dm + dm.T
        v1 = vresp(dm1)
        v1vo = numpy.empty_like(mo1)
        for i, x in enumerate(v1):
            v1vo[i] = reduce(numpy.dot, (mo_coeff.T, x, mocc))
        return v1vo
    return fx

In [165]:
fx = gen_vind(mf, mo_coeff, mo_occ)

In [185]:
mo1, e1 = cphf.solve(fx, mo_energy, mo_occ, h1vo, s1vo)
mo1 = numpy.einsum('pq,qi->pi', mo_coeff, mo1).reshape(nao,nocc)
dm1 = numpy.einsum('pi,qi->pq', mo1, mocc)*4

In [188]:
make_dP(mf,mo1)

array([[-2.40558754e-01,  1.07636705e+00, -3.09961397e+00,
         3.36931804e-01, -2.98018597e-01,  4.53974894e-01,
         1.51772437e-01,  9.23010901e-01, -1.47264570e+00,
         6.01935034e-01, -4.04216536e-01,  3.40049952e+00,
         3.45772779e-01, -7.64354232e-01,  1.85032880e-01,
         2.71501805e-01,  1.91307337e-01, -1.39041115e+00],
       [ 1.07636705e+00, -2.62323461e+00,  4.79775323e-01,
        -7.66982154e-01, -6.03760094e-01,  1.27345129e+00,
        -2.78897648e-01, -1.18823593e+00,  2.27897429e+00,
        -2.74750592e+00,  1.49591475e+00, -2.22507256e+00,
        -7.45662249e-01, -3.06187557e-01, -4.78559610e-01,
        -5.68375793e-01, -4.90586041e-01,  9.74557910e-01],
       [-3.09961397e+00,  4.79775323e-01,  1.28091885e+01,
         9.53734998e-01,  1.80442636e+00, -4.67023197e+00,
         6.98424564e-01, -6.19127942e-01,  2.23577504e+00,
         9.86240622e+00, -8.49176462e-01, -9.22001511e+00,
         9.62700152e-01,  2.43259786e+00,  5.31110248e

In [186]:
numpy.einsum('pq,pq', h1_ao, dP_fd),numpy.einsum('pq,pq', h1_ao, dm1)

(-0.7066089954695508, 0.4353498064720487)